In [1]:
import os

In [ ]:
for file in os.listdir("../datasets/taillard/"):
    with open("../datasets/taillard/" + file, "r") as f:
        lines = f.readlines()
        n_jobs, n_machines, _ = map(int, lines[0].split())
        duration_matrix = [[0 for _ in range(n_jobs)] for _ in range(n_machines)]
        for i in range(1, len(lines)):
            lines[i] = " ".join(lines[i].split()[4:]) + "\n"
        duration_matrix = [
            [int(x) for i, x in enumerate(line.split()) if i % 2] for line in lines[1:]
        ]
        machines = [
            [int(x) for i, x in enumerate(line.split()) if i % 2 == 0]
            for line in lines[1:]
        ]
    with open("../datasets/taillard/" + file, "w") as f:
        f.write(str(n_jobs) + " " + str(n_machines) + "\n")
        for duration in duration_matrix:
            f.write(" ".join(map(str, duration)) + "\n")
        f.write("\n")
        for machine in machines:
            f.write(" ".join(map(str, machine)) + "\n")

In [11]:
for file in os.listdir("../datasets/dmu/"):
    with open("../datasets/dmu/" + file, "r") as f:
        lines = f.readlines()
        n_jobs, n_machines = map(int, lines[0].split())
        duration_matrix = [[0 for _ in range(n_jobs)] for _ in range(n_machines)]
        for i in range(1, len(lines)):
            lines[i] = " ".join(lines[i].split()) + "\n"
        duration_matrix = [
            [int(x) for i, x in enumerate(line.split()) if i % 2] for line in lines[1:]
        ]
        machines = [
            [int(x) + 1 for i, x in enumerate(line.split()) if i % 2 == 0]
            for line in lines[1:]
        ]
        with open("../datasets/dmu/" + file, "w") as f:
            f.write(str(n_jobs) + " " + str(n_machines) + "\n")
            for duration in duration_matrix:
                f.write(" ".join(map(str, duration)) + "\n")
            f.write("\n")
            for machine in machines:
                f.write(" ".join(map(str, machine)) + "\n")
        # print(n_jobs, n_machines)
        # for duration in duration_matrix:
        #     print(*duration)
        # print()
        # for machine in machines:
        #     print(*machine)
        # break

In [5]:
import glob
import shutil

# Get a list of all CSV files in the source directory
csv_files = glob.glob("../datasets/*.csv")

# Move each CSV file to the destination directory
for file in csv_files:
    shutil.move(file, "../results/")

